In [1]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel

In [2]:
# Mounting Google Drive to access files stored in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/NLP_Project/IMDB_Dataset_Preprocessed.csv')

In [4]:
# Load DistilBERT model and tokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

# Function to get embeddings
def get_model_embeddings(tokenizer, model, review):
    inputs = tokenizer(review, return_tensors="tf", max_length=512, truncation=True, padding='max_length')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = tf.reduce_mean(last_hidden_states, axis=1)
    return sentence_embedding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [5]:
try:
  X_distilbert = np.load('/content/drive/MyDrive/NLP_Project/X_distilbert.npy')
  print("DistilBERT Feature Shape:", X_distilbert.shape)
except:
  review  = df['cleaned_review'][0]
  X_distilbert = np.array([get_model_embeddings(distilbert_tokenizer, distilbert_model, review)[0]])
  np.save('/content/drive/MyDrive/NLP_Project/X_distilbert.npy', X_distilbert)
# Convert reviews to embeddings
for i in range(X_distilbert.shape[0], len(df['cleaned_review'])):
  review  = df['cleaned_review'][i]
  X_distilbert = np.append(X_distilbert, np.array([get_model_embeddings(distilbert_tokenizer, distilbert_model, review)[0]]), axis=0)
  np.save('/content/drive/MyDrive/NLP_Project/X_distilbert.npy', X_distilbert)

DistilBERT Feature Shape: (50000, 768)
